<a href="https://colab.research.google.com/github/MK316/Getpp24/blob/main/Sentiment0930.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌱 GetPP-Sentiment analysis (0821 9PM)

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/Getpp24/main/data/resultall0815-light.csv"

# Step 1: Read the dataframe from the provided URL
df = pd.read_csv(url)

In [ ]:
df.head()

Check cells of 'Bepp' and 'Getpp' whether they are in lists

In [ ]:
import pandas as pd

# Check the datatype of each cell in 'Bepp' and display row numbers if the cell is not a list
non_list_rows = df[~df['Bepp'].apply(lambda x: isinstance(x, list))].index

# Output the row numbers where 'Bepp' is not a list
if not non_list_rows.empty:
    print("Rows where 'Bepp' is not a list:")
    print(non_list_rows.tolist())
else:
    print("All cells in the 'Bepp' column are lists.")


## 'Bepp' cell as lists

In [ ]:
import pandas as pd
import ast


# Function to check and convert strings that look like lists into actual lists
def convert_to_list_if_needed(cell):
    if isinstance(cell, str):
        try:
            # Try to convert it to a list
            return ast.literal_eval(cell)
        except (ValueError, SyntaxError):
            # If it fails, return the original string
            return cell
    return cell

# Apply the function to both 'Bepp' and 'Getpp' columns
df['Bepp'] = df['Bepp'].apply(convert_to_list_if_needed)
df['Getpp'] = df['Getpp'].apply(convert_to_list_if_needed)

# Check the datatype of each cell in 'Bepp' and 'Getpp' and display row numbers if the cell is not a list
non_list_bepp_rows = df[~df['Bepp'].apply(lambda x: isinstance(x, list))].index
non_list_getpp_rows = df[~df['Getpp'].apply(lambda x: isinstance(x, list))].index

# Output the row numbers where 'Bepp' is not a list
if not non_list_bepp_rows.empty:
    print("Rows where 'Bepp' is not a list:")
    print(non_list_bepp_rows.tolist())
else:
    print("All cells in the 'Bepp' column are lists.")

# Output the row numbers where 'Getpp' is not a list
if not non_list_getpp_rows.empty:
    print("Rows where 'Getpp' is not a list:")
    print(non_list_getpp_rows.tolist())
else:
    print("All cells in the 'Getpp' column are lists.")


Bepp-spoken vs. Bepp-written: two lists of sentences

In [ ]:
import pandas as pd

# Check DataFrame
# print("Original DataFrame:")
# print(df)

# Filter and combine sentence lists based on 'register'
bepp_spoken = sum(df[df['register'] == 'Spoken']['Bepp'], [])
bepp_written = sum(df[df['register'] == 'Written']['Bepp'], [])

getpp_spoken = sum(df[df['register'] == 'Spoken']['Getpp'], [])
getpp_written = sum(df[df['register'] == 'Written']['Getpp'], [])
# Display the combined lists
print("\nCombined Sentences for Spoken Register: Be pp")
print(len(bepp_spoken), bepp_spoken)
print("\nCombined Sentences for Written Register: Be pp")
print(len(bepp_written), bepp_written)

print("\nCombined Sentences for Spoken Register: Get pp")
print(len(getpp_spoken), getpp_spoken)
print("\nCombined Sentences for Written Register: Get pp")
print(len(getpp_written), getpp_written)


# Sentiment analysis with 4 lists

+ bepp_spoken, _written
+ getpp_spoken, _written

In [ ]:
!pip install nltk
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
import nltk
nltk.download('opinion_lexicon')
nltk.data.find('corpora/opinion_lexicon')



In [ ]:
# Load the positive and negative word lists from the uploaded files with explicit encoding
positive_words = set([word.strip() for word in open('/content/positive-words.txt', encoding='ISO-8859-1')])
negative_words = set([word.strip() for word in open('/content/negative-words.txt', encoding='ISO-8859-1')])

# Print a sample of the words
print("Sample of positive words:", list(positive_words)[:5])
print("Sample of negative words:", list(negative_words)[:5])



In [ ]:
verbs = ['destroyed', 'stolen', 'written', 'approved']

for verb in verbs:
    print(f"Verb '{verb}' is positive: {verb in positive_words}")
    print(f"Verb '{verb}' is negative: {verb in negative_words}")


In [ ]:
# Manually add key verbs to the lexicon if necessary
positive_words.update(['approved', 'written', 'handled'])
negative_words.update(['destroyed', 'stolen'])

# Re-run the analysis


In [ ]:
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')

# Function to identify passive constructions and extract the past participle verbs
def extract_passive_verbs(sentence):
    doc = nlp(sentence)
    verbs = []

    for token in doc:
        # Identify passive constructions via the dependency labels 'auxpass' and 'nsubjpass'
        if token.dep_ == 'auxpass':  # Passive auxiliary (e.g., "was", "is", "get")
            # Look for the main verb which should be a past participle in a passive
            for child in token.head.children:
                if child.dep_ == 'acl' or child.pos_ == 'VERB':  # Extract past participle verb
                    verbs.append(child.lemma_)  # Use lemma to normalize the verb

    return verbs

# Function to analyze the sentiment of the verb tokens extracted from passive constructions
def analyze_verb_sentiment(verbs, positive_words, negative_words):
    sentiment_scores = {'positive': 0, 'negative': 0, 'neutral': 0}

    for verb in verbs:
        if verb in positive_words:
            sentiment_scores['positive'] += 1  # Count each token occurrence
        elif verb in negative_words:
            sentiment_scores['negative'] += 1
        else:
            sentiment_scores['neutral'] += 1

    return sentiment_scores

# Function to process each list of sentences (token-based approach)
def analyze_sentences(sentences, positive_words, negative_words):
    all_verbs = []

    for sentence in sentences:
        verbs = extract_passive_verbs(sentence)
        all_verbs.extend(verbs)  # Collect all verbs across sentences

    return analyze_verb_sentiment(all_verbs, positive_words, negative_words)


# Perform sentiment analysis for each list
bepp_spoken_sentiment = analyze_sentences(bepp_spoken, positive_words, negative_words)
bepp_written_sentiment = analyze_sentences(bepp_written, positive_words, negative_words)
getpp_spoken_sentiment = analyze_sentences(getpp_spoken, positive_words, negative_words)
getpp_written_sentiment = analyze_sentences(getpp_written, positive_words, negative_words)

# Display the results
print("BePP Spoken Sentiment:", bepp_spoken_sentiment)
print("BePP Written Sentiment:", bepp_written_sentiment)
print("GetPP Spoken Sentiment:", getpp_spoken_sentiment)
print("GetPP Written Sentiment:", getpp_written_sentiment)


Updated codes using vaderSentiment (word lists)

+ If you want to automatically handle a wider range of verbs without manually modifying the lexicon, you can use a pre-built sentiment analysis tool like TextBlob or VADER. These models are more comprehensive and can classify sentiment for a broader range of words.

+ Hutto, C. J., & Gilbert, E. E. (2014). VADER: A parsimonious rule-based model for sentiment analysis of social media text. Proceedings of the International AAAI Conference on Weblogs and Social Media, 216–225.

In [ ]:
!pip install vaderSentiment


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment using VADER
def analyze_sentences_vader(sentences):
    sentiment_scores = {'positive': 0, 'negative': 0, 'neutral': 0}

    for sentence in sentences:
        score = analyzer.polarity_scores(sentence)
        if score['compound'] >= 0.05:
            sentiment_scores['positive'] += 1
        elif score['compound'] <= -0.05:
            sentiment_scores['negative'] += 1
        else:
            sentiment_scores['neutral'] += 1

    return sentiment_scores

# Test with dummy sentences
bepp_spoken_sentiment = analyze_sentences_vader(bepp_spoken)
bepp_written_sentiment = analyze_sentences_vader(bepp_written)
getpp_spoken_sentiment = analyze_sentences_vader(getpp_spoken)
getpp_written_sentiment = analyze_sentences_vader(getpp_written)

print("BePP Spoken Sentiment (VADER):", bepp_spoken_sentiment)
print("BePP Written Sentiment (VADER):", bepp_written_sentiment)
print("GetPP Spoken Sentiment (VADER):", getpp_spoken_sentiment)
print("GetPP Written Sentiment (VADER):", getpp_written_sentiment)


### Plot: sentiment results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data from the results
categories = ['BePP Spoken', 'BePP Written', 'GetPP Spoken', 'GetPP Written']
positive = [3542, 5020, 120, 54]
negative = [2579, 4011, 118, 34]
neutral = [3426, 3659, 133, 44]

# Create a bar plot for the sentiments
def plot_sentiment_bar():
    bar_width = 0.2
    index = np.arange(len(categories))

    # Create the bar chart
    plt.figure(figsize=(10, 6))
    plt.bar(index, positive, bar_width, label='Positive', color='g')
    plt.bar(index + bar_width, negative, bar_width, label='Negative', color='r')
    plt.bar(index + 2 * bar_width, neutral, bar_width, label='Neutral', color='b')

    # Add labels and formatting
    plt.xlabel('Categories')
    plt.ylabel('Counts')
    plt.title('Sentiment Analysis for BePP and GetPP (Spoken vs Written)')
    plt.xticks(index + bar_width, categories)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Create a stacked bar plot for the sentiments
def plot_sentiment_stacked_bar():
    bar_width = 0.5
    index = np.arange(len(categories))

    # Create the stacked bar chart
    plt.figure(figsize=(10, 6))
    plt.bar(index, positive, bar_width, label='Positive', color='g')
    plt.bar(index, negative, bar_width, bottom=positive, label='Negative', color='r')
    plt.bar(index, neutral, bar_width, bottom=np.array(positive) + np.array(negative), label='Neutral', color='b')

    # Add labels and formatting
    plt.xlabel('Categories')
    plt.ylabel('Counts')
    plt.title('Stacked Sentiment Analysis for BePP and GetPP (Spoken vs Written)')
    plt.xticks(index, categories)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the functions to generate the plots
plot_sentiment_bar()
plot_sentiment_stacked_bar()


## Normalized sentiment plot (0929)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data from the results
categories = ['BePP Spoken', 'BePP Written', 'GetPP Spoken', 'GetPP Written']
positive = np.array([3542, 5020, 120, 54])
negative = np.array([2579, 4011, 118, 34])
neutral = np.array([3426, 3659, 133, 44])

# Calculate the total number of sentiments for each category
total = positive + negative + neutral

# Normalize the sentiment counts to get proportions
positive_proportion = positive / total
negative_proportion = negative / total
neutral_proportion = neutral / total

# Create a stacked bar plot for the proportional sentiments
def plot_proportional_stacked_bar():
    bar_width = 0.5
    index = np.arange(len(categories))

    # Create the stacked bar chart with proportions
    plt.figure(figsize=(10, 6))
    plt.bar(index, positive_proportion, bar_width, label='Positive', color='g')
    plt.bar(index, negative_proportion, bar_width, bottom=positive_proportion, label='Negative', color='r')
    plt.bar(index, neutral_proportion, bar_width, bottom=positive_proportion + negative_proportion, label='Neutral', color='b')

    # Add labels and formatting
    plt.xlabel('Categories')
    plt.ylabel('Proportion')
    plt.title('Proportional Sentiment Analysis for BePP and GetPP (Spoken vs Written)')
    plt.xticks(index, categories)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function to generate the plot
plot_proportional_stacked_bar()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data from the results
categories = ['BePP Spoken', 'BePP Written', 'GetPP Spoken', 'GetPP Written']
positive = [3542, 5020, 120, 54]
negative = [2579, 4011, 118, 34]
neutral = [3426, 3659, 133, 44]

# Calculate the total number of sentiments for each category
total = np.array(positive) + np.array(negative) + np.array(neutral)

# Normalize the sentiment counts to get proportions
positive_proportion = np.array(positive) / total
negative_proportion = np.array(negative) / total
neutral_proportion = np.array(neutral) / total

# Create a grouped bar plot
def plot_grouped_bar():
    bar_width = 0.25
    index = np.arange(len(categories))

    # Create the grouped bar chart
    plt.figure(figsize=(12, 6))
    plt.bar(index, positive_proportion, bar_width, label='Positive', color='g')
    plt.bar(index + bar_width, negative_proportion, bar_width, label='Negative', color='r')
    plt.bar(index + 2 * bar_width, neutral_proportion, bar_width, label='Neutral', color='b')

    # Add labels and formatting
    plt.xlabel('Categories')
    plt.ylabel('Proportion')
    plt.title('Proportional Sentiment Analysis by Register and Construction Type')
    plt.xticks(index + bar_width, categories)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function to generate the grouped bar plot
plot_grouped_bar()


## Heatmap

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Data from the results
categories = ['BePP Spoken', 'BePP Written', 'GetPP Spoken', 'GetPP Written']
positive = [3542, 5020, 120, 54]
negative = [2579, 4011, 118, 34]
neutral = [3426, 3659, 133, 44]

# Create a matrix of sentiment proportions
sentiment_matrix = np.array([
    [3542, 5020, 120, 54],  # Positive
    [2579, 4011, 118, 34],  # Negative
    [3426, 3659, 133, 44]   # Neutral
])

# Normalize the matrix to show proportions
total = sentiment_matrix.sum(axis=0)
sentiment_matrix_proportions = sentiment_matrix / total

# Create a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(sentiment_matrix_proportions, annot=True, cmap='coolwarm', xticklabels=categories, yticklabels=['Positive', 'Negative', 'Neutral'])
plt.title('Sentiment Proportions by Register and Construction Type')
plt.xlabel('Category')
plt.ylabel('Sentiment Type')
plt.savefig('Sentiment-heatmap.png')
plt.tight_layout()
plt.show()


# Chi-squared test with sentiment results

In [ ]:
from scipy.stats import chi2_contingency
import numpy as np

# Observed data (in contingency table format)
observed_data = np.array([
    [3542, 5020, 120, 54],  # Positive counts
    [2579, 4011, 118, 34],  # Negative counts
    [3426, 3659, 133, 44]   # Neutral counts
])

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(observed_data)

# Print the results
print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of Freedom: {dof}")
print(f"Expected frequencies: \n{expected}")


## Residuals and contributions

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

# Data from the results
observed = np.array([[3542, 5020, 120, 54],
                     [2579, 4011, 118, 34],
                     [3426, 3659, 133, 44]])

# Perform Chi-square test
chi2_stat, p_val, dof, expected = chi2_contingency(observed)

# Calculate residuals and contributions
residuals = (observed - expected) / np.sqrt(expected)
contributions = (observed - expected)**2 / expected

# Create a dataframe to display results
categories = ['BePP Spoken', 'BePP Written', 'GetPP Spoken', 'GetPP Written']
index = ['Positive', 'Negative', 'Neutral']

# Convert arrays to DataFrames for better presentation
observed_df = pd.DataFrame(observed, columns=categories, index=index)
expected_df = pd.DataFrame(expected, columns=categories, index=index)
residuals_df = pd.DataFrame(residuals, columns=categories, index=index)
contributions_df = pd.DataFrame(contributions, columns=categories, index=index)

# Display results
print("Observed Frequencies:\n", observed_df)
print("\nExpected Frequencies:\n", expected_df)
print("\nResiduals:\n", residuals_df)
print("\nChi-square Contributions:\n", contributions_df)

# Optional: visualize residuals as a heatmap
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.heatmap(residuals_df, annot=True, cmap="coolwarm", center=0)
plt.title("Residuals Heatmap")
plt.show()

plt.figure(figsize=(10, 6))
sns.heatmap(contributions_df, annot=True, cmap="YlGnBu")
plt.title("Chi-square Contributions Heatmap")
plt.show()


# Part II. Sentiment analysis with past participles (maybe this is a better way)

[1] data to read and get 4 sets of lists

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/Getpp24/main/data/resultall0815-light.csv"

# Step 1: Read the dataframe from the provided URL
df = pd.read_csv(url)

import pandas as pd
import ast


# Function to check and convert strings that look like lists into actual lists
def convert_to_list_if_needed(cell):
    if isinstance(cell, str):
        try:
            # Try to convert it to a list
            return ast.literal_eval(cell)
        except (ValueError, SyntaxError):
            # If it fails, return the original string
            return cell
    return cell

# Apply the function to both 'Bepp' and 'Getpp' columns
df['Bepp'] = df['Bepp'].apply(convert_to_list_if_needed)
df['Getpp'] = df['Getpp'].apply(convert_to_list_if_needed)

# Check the datatype of each cell in 'Bepp' and 'Getpp' and display row numbers if the cell is not a list
non_list_bepp_rows = df[~df['Bepp'].apply(lambda x: isinstance(x, list))].index
non_list_getpp_rows = df[~df['Getpp'].apply(lambda x: isinstance(x, list))].index

# Output the row numbers where 'Bepp' is not a list
if not non_list_bepp_rows.empty:
    print("Rows where 'Bepp' is not a list:")
    print(non_list_bepp_rows.tolist())
else:
    print("All cells in the 'Bepp' column are lists.")

# Output the row numbers where 'Getpp' is not a list
if not non_list_getpp_rows.empty:
    print("Rows where 'Getpp' is not a list:")
    print(non_list_getpp_rows.tolist())
else:
    print("All cells in the 'Getpp' column are lists.")

import pandas as pd

# Check DataFrame
# print("Original DataFrame:")
# print(df)

# Filter and combine sentence lists based on 'register'
bepp_spoken = sum(df[df['register'] == 'Spoken']['Bepp'], [])
bepp_written = sum(df[df['register'] == 'Written']['Bepp'], [])

getpp_spoken = sum(df[df['register'] == 'Spoken']['Getpp'], [])
getpp_written = sum(df[df['register'] == 'Written']['Getpp'], [])
# Display the combined lists
print("\nCombined Sentences for Spoken Register: Be pp")
print(len(bepp_spoken), bepp_spoken)
print("\nCombined Sentences for Written Register: Be pp")
print(len(bepp_written), bepp_written)

print("\nCombined Sentences for Spoken Register: Get pp")
print(len(getpp_spoken), getpp_spoken)
print("\nCombined Sentences for Written Register: Get pp")
print(len(getpp_written), getpp_written)


---
# Restart from here

Try quadrogram with bepp_spoken

[1] quadrogram be pp => csv

In [ ]:
import pandas as pd
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')


# Create a DataFrame
df_bepp_spoken = pd.DataFrame(bepp_spoken, columns=['Sentences'])

# Lemma forms of 'be' to match any tense or form including contractions
be_forms = {'be', 'is', 'am', 'are', "'s", 'was', 'were', 'been', 'being'}

# Function to extract quadrograms starting with any 'be' form verb
def extract_quadrograms(sentence):
    doc = nlp(sentence)
    quadrograms = []

    # Loop over tokens and extract quadrograms starting with any 'be' verb
    for i in range(len(doc) - 3):
        token1, token2, token3, token4 = doc[i], doc[i+1], doc[i+2], doc[i+3]

        # Check if the first token is a form of 'be'
        if token1.lemma_ in be_forms:
            quadrogram = (token1.text, token2.text, token3.text, token4.text)
            quadrograms.append(quadrogram)

    return quadrograms

# List to hold all extracted quadrograms
quadrogram_list = []

# Extract quadrograms for each sentence in the DataFrame
for sentence in df_bepp_spoken['Sentences']:
    quadrograms = extract_quadrograms(sentence)
    quadrogram_list.extend(quadrograms)

# Create a DataFrame from the extracted quadrograms
df_quadrograms = pd.DataFrame(quadrogram_list, columns=['1st', '2nd', '3rd', '4th'])

# Save the DataFrame to a CSV file (optional)
df_quadrograms.to_csv('quadrograms-bepp-spoken.csv', index=False)

# Display the first few rows of the DataFrame
print(df_quadrograms.head())


[2] quadrogram be_written => csv

In [ ]:
import pandas as pd
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')


# Create a DataFrame
df_bepp_written = pd.DataFrame(bepp_written, columns=['Sentences'])

# Lemma forms of 'be' to match any tense or form including contractions
be_forms = {'be', 'is', 'am', 'are', "'s", 'was', 'were', 'been', 'being'}

# Function to extract quadrograms starting with any 'be' form verb
def extract_quadrograms(sentence):
    doc = nlp(sentence)
    quadrograms = []

    # Loop over tokens and extract quadrograms starting with any 'be' verb
    for i in range(len(doc) - 3):
        token1, token2, token3, token4 = doc[i], doc[i+1], doc[i+2], doc[i+3]

        # Check if the first token is a form of 'be'
        if token1.lemma_ in be_forms:
            quadrogram = (token1.text, token2.text, token3.text, token4.text)
            quadrograms.append(quadrogram)

    return quadrograms

# List to hold all extracted quadrograms
quadrogram_list = []

# Extract quadrograms for each sentence in the DataFrame
for sentence in df_bepp_written['Sentences']:
    quadrograms = extract_quadrograms(sentence)
    quadrogram_list.extend(quadrograms)

# Create a DataFrame from the extracted quadrograms
df_quadrograms = pd.DataFrame(quadrogram_list, columns=['1st', '2nd', '3rd', '4th'])

# Save the DataFrame to a CSV file (optional)
df_quadrograms.to_csv('quadrograms-bepp-written.csv', index=False)

# Display the first few rows of the DataFrame
print(df_quadrograms.head())


[3] quadrogram get spoken

In [ ]:
import pandas as pd
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')


# Create a DataFrame
df_getpp_spoken = pd.DataFrame(getpp_spoken, columns=['Sentences'])

# Lemma forms of 'be' to match any tense or form including contractions
get_forms = {'get', 'got', 'getting'}

# Function to extract quadrograms starting with any 'be' form verb
def extract_quadrograms(sentence):
    doc = nlp(sentence)
    quadrograms = []

    # Loop over tokens and extract quadrograms starting with any 'be' verb
    for i in range(len(doc) - 3):
        token1, token2, token3, token4 = doc[i], doc[i+1], doc[i+2], doc[i+3]

        # Check if the first token is a form of 'be'
        if token1.lemma_ in get_forms:
            quadrogram = (token1.text, token2.text, token3.text, token4.text)
            quadrograms.append(quadrogram)

    return quadrograms

# List to hold all extracted quadrograms
quadrogram_list = []

# Extract quadrograms for each sentence in the DataFrame
for sentence in df_getpp_spoken['Sentences']:
    quadrograms = extract_quadrograms(sentence)
    quadrogram_list.extend(quadrograms)

# Create a DataFrame from the extracted quadrograms
df_quadrograms = pd.DataFrame(quadrogram_list, columns=['1st', '2nd', '3rd', '4th'])

# Save the DataFrame to a CSV file (optional)
df_quadrograms.to_csv('quadrograms-getpp-spoken.csv', index=False)

# Display the first few rows of the DataFrame
print(df_quadrograms.head())


[4] quadrogram get written

In [ ]:
import pandas as pd
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')


# Create a DataFrame
df_getpp_written = pd.DataFrame(getpp_written, columns=['Sentences'])

# Lemma forms of 'be' to match any tense or form including contractions
get_forms = {'get', 'got', 'getting'}

# Function to extract quadrograms starting with any 'be' form verb
def extract_quadrograms(sentence):
    doc = nlp(sentence)
    quadrograms = []

    # Loop over tokens and extract quadrograms starting with any 'be' verb
    for i in range(len(doc) - 3):
        token1, token2, token3, token4 = doc[i], doc[i+1], doc[i+2], doc[i+3]

        # Check if the first token is a form of 'be'
        if token1.lemma_ in get_forms:
            quadrogram = (token1.text, token2.text, token3.text, token4.text)
            quadrograms.append(quadrogram)

    return quadrograms

# List to hold all extracted quadrograms
quadrogram_list = []

# Extract quadrograms for each sentence in the DataFrame
for sentence in df_getpp_written['Sentences']:
    quadrograms = extract_quadrograms(sentence)
    quadrogram_list.extend(quadrograms)

# Create a DataFrame from the extracted quadrograms
df_quadrograms = pd.DataFrame(quadrogram_list, columns=['1st', '2nd', '3rd', '4th'])

# Save the DataFrame to a CSV file (optional)
df_quadrograms.to_csv('quadrograms-getpp-written.csv', index=False)

# Display the first few rows of the DataFrame
print(df_quadrograms.head())


### To sentences (dataframe): 4 files saved

+ no need to run below

In [ ]:
import pandas as pd

# Create a DataFrame with 'Sentences' as the column name
df_bepp_spoken = pd.DataFrame(bepp_spoken, columns=['Sentences'])

# Save the DataFrame to a CSV file
df_bepp_spoken.to_csv('bepp_spoken_sentences.csv', index=False)

# Output the first few rows of the DataFrame as a check
print(df_bepp_spoken.head())


The following code processes:

### Explanation:
+ Lemmatization:
We use spaCy's lemmatization (with token.lemma_) to handle all forms of "be" and "get" in different tenses and aspects. This also covers contractions, as spaCy normalizes forms like "he's" to "be" and "he's gotten" to "get".
+ Past Participle Identification:
We identify the past participle (VBN) that is dependent on the auxiliary verb ("be" or "get"). This ensures that we correctly extract the verb used in the passive construction.
+ Handling All Forms of "Be" and "Get":
The function handles all variations of "be" (e.g., "is", "was", "were", "been", "being") and "get" (e.g., "got", "getting"). These forms are normalized to "be" and "get" using lemmatization.

test

In [ ]:
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')

# Function to extract trigrams for 'be' passives using dependency parsing
def extract_be_passive_trigrams(sentences):
    trigrams = []

    # Lemma forms of 'be' to match any tense or form
    be_forms = {'be', 'is', 'am', 'are', 'was', 'were', 'been', 'being'}

    for sentence in sentences:
        doc = nlp(sentence)
        for token in doc:
            # Check if the token is a form of 'be' and is an auxiliary verb
            if token.lemma_ in be_forms and token.dep_ == 'aux':
                head = token.head  # The head verb the auxiliary is attached to
                if head.tag_ == 'VBN':  # Past participle form
                    # Filter out irrelevant tokens (punctuation, subjects, etc.)
                    children = [child.text for child in head.children if child.dep_ in {'advmod', 'aux', 'neg'}]
                    trigram = (token.text, ' '.join(children), head.text)
                    trigrams.append(trigram)

    return trigrams

# Function to extract trigrams for 'get' passives using dependency parsing
def extract_get_passive_trigrams(sentences):
    trigrams = []

    # Lemma forms of 'get' to match any tense or form
    get_forms = {'get', 'got', 'getting'}

    for sentence in sentences:
        doc = nlp(sentence)
        for token in doc:
            # Check if the token is a form of 'get' and is an auxiliary verb
            if token.lemma_ in get_forms and token.dep_ == 'aux':
                head = token.head  # The head verb the auxiliary is attached to
                if head.tag_ == 'VBN':  # Past participle form
                    # Filter out irrelevant tokens (punctuation, subjects, etc.)
                    children = [child.text for child in head.children if child.dep_ in {'advmod', 'aux', 'neg'}]
                    trigram = (token.text, ' '.join(children), head.text)
                    trigrams.append(trigram)

    return trigrams


# Example usage with separate datasets
bepp_spoken1 = [
    "The building was completely destroyed.",
    "The car had been quickly stolen.",
    "They are being carefully arrested."
]

bepp_written1 = [
    "The report has been written by the committee.",
    "The document was approved."
]

getpp_spoken1 = [
    "He got promoted last year.",
    "The award is getting given to John.",
    "The problem got solved quickly."
]

getpp_written1 = [
    "The issue will get handled by the team.",
    "The task got completed."
]

# Extract trigrams for 'be' passives
bepp_spoken_trigrams = extract_be_passive_trigrams(bepp_spoken1)
bepp_written_trigrams = extract_be_passive_trigrams(bepp_written1)

# Extract trigrams for 'get' passives
getpp_spoken_trigrams = extract_get_passive_trigrams(getpp_spoken1)
getpp_written_trigrams = extract_get_passive_trigrams(getpp_written1)

# Output the extracted trigrams
print("BePP Spoken Trigrams:", bepp_spoken_trigrams)
print("BePP Written Trigrams:", bepp_written_trigrams)
print("GetPP Spoken Trigrams:", getpp_spoken_trigrams)
print("GetPP Written Trigrams:", getpp_written_trigrams)


In [ ]:
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')

# Function to extract trigrams where the first word is a form of 'be'
def extract_passive_trigrams(sentences):
    trigrams = []

    # Lemma forms of 'be' and 'get' to match any tense or form
    be_forms = {'be', 'is', 'am', 'are', 'was', 'were', 'been', 'being', 'get', 'got', 'getting'}

    for sentence in sentences:
        doc = nlp(sentence)
        tokens = [token for token in doc]  # Tokenize the sentence

        # Use a sliding window to extract trigrams (3-word sequences)
        for i in range(len(tokens) - 2):
            token1, token2, token3 = tokens[i], tokens[i+1], tokens[i+2]

            # Check if the first token is a form of 'be' or 'get', and the third token is a past participle
            if token1.lemma_ in be_forms and token3.tag_ == 'VBN':
                trigram = (token1.text, token2.text, token3.text)
                trigrams.append(trigram)

    return trigrams


# Extract trigrams for each dataset
bepp_spoken_trigrams = extract_passive_trigrams(bepp_spoken)
bepp_written_trigrams = extract_passive_trigrams(bepp_written)
getpp_spoken_trigrams = extract_passive_trigrams(getpp_spoken)
getpp_written_trigrams = extract_passive_trigrams(getpp_written)

# Output the extracted trigrams
print("BePP Spoken Trigrams:", bepp_spoken_trigrams)
print("BePP Written Trigrams:", bepp_written_trigrams)
print("GetPP Spoken Trigrams:", getpp_spoken_trigrams)
print("GetPP Written Trigrams:", getpp_written_trigrams)


Save the output as dataframe

In [ ]:
import pandas as pd

# Convert list of tuples to a DataFrame
df_bepp_spoken = pd.DataFrame(bepp_spoken_trigrams, columns=['1st', '2nd', '3rd'])
df_bepp_written = pd.DataFrame(bepp_spoken_trigrams, columns=['1st', '2nd', '3rd'])
df_getpp_spoken = pd.DataFrame(getpp_spoken_trigrams, columns=['1st', '2nd', '3rd'])
df_getpp_written = pd.DataFrame(getpp_spoken_trigrams, columns=['1st', '2nd', '3rd'])

# Display the DataFrame
print(df_bepp_spoken)

# Save the DataFrame to a CSV file
df_bepp_spoken.to_csv('bepp_spoken_trigrams.csv', index=False)
df_bepp_written.to_csv('bepp_written_trigrams.csv', index=False)
df_getpp_spoken.to_csv('getpp_spoken_trigrams.csv', index=False)
df_getpp_written.to_csv('getpp_written_trigrams.csv', index=False)

# Sentiment analysis with polarity

In [ ]:
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')

# Function to extract past participles following any form of 'be' or 'get'
def extract_past_participles(sentences, auxiliary_lemmas):
    past_participles = []

    for sentence in sentences:
        doc = nlp(sentence)
        for token in doc:
            # Check if the token's lemma is 'be' or 'get' (covers all tenses/aspects including contractions)
            if token.lemma_ in auxiliary_lemmas and token.dep_ == 'aux':
                # Find the main verb (past participle) that depends on the auxiliary
                for child in token.head.children:
                    if child.pos_ == 'VERB' and child.tag_ == 'VBN':  # Past participle form
                        past_participles.append(child.lemma_)  # Use lemma to normalize the verb

    return past_participles


# Extract past participles after 'be' and 'get'
bepp_spoken = extract_past_participles(bepp_spoken, ['be'])
bepp_written = extract_past_participles(bepp_written, ['be'])
getpp_spoken = extract_past_participles(getpp_spoken, ['get'])
getpp_written = extract_past_participles(getpp_written, ['get'])

# Output the extracted past participles
print("BePP Spoken Past Participles:", bepp_spoken)
print("BePP Written Past Participles:", bepp_written)
print("GetPP Spoken Past Participles:", getpp_spoken)
print("GetPP Written Past Participles:", getpp_written)


In [ ]:
len(bepp_spoken)

## Types of participles

In [ ]:
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')

# Function to extract past participles following any form of 'be' or 'get'
def extract_past_participles(sentences, auxiliary_lemmas):
    past_participles = []

    for sentence in sentences:
        doc = nlp(sentence)
        for token in doc:
            # Check if the token's lemma is 'be' or 'get'
            if token.lemma_ in auxiliary_lemmas:
                # Find the main verb (past participle) that depends on the auxiliary OR "get" as a main verb
                for child in token.head.children:
                    if child.pos_ == 'VERB' and child.tag_ == 'VBN':  # Past participle form
                        past_participles.append(child.lemma_)  # Use lemma to normalize the verb

                # Handle the case where "get" is the main verb (root of the sentence)
                if token.lemma_ == 'get' and token.pos_ == 'VERB' and token.tag_ == 'VBN':
                    past_participles.append(token.head.lemma_)

    return past_participles


# Extract past participles after 'be' and 'get'
bepp_spoken = extract_past_participles(bepp_spoken, ['be'])
bepp_written = extract_past_participles(bepp_written, ['be'])
getpp_spoken = extract_past_participles(getpp_spoken, ['get'])
getpp_written = extract_past_participles(getpp_written, ['get'])

# Output the extracted past participles
print("BePP Spoken Past Participles:", bepp_spoken)
print("BePP Written Past Participles:", bepp_written)
print("GetPP Spoken Past Participles:", getpp_spoken)
print("GetPP Written Past Participles:", getpp_written)


In [ ]:
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')

# Function to extract past participles for passive constructions
def extract_past_participles(sentences, auxiliary_lemmas):
    past_participles = []

    for sentence in sentences:
        print(f"\nProcessing sentence: {sentence}")
        doc = nlp(sentence)

        # Print the token, lemma, pos, tag, and dependency relation for each token
        for token in doc:
            print(f"Token: {token.text}, Lemma: {token.lemma_}, POS: {token.pos_}, Tag: {token.tag_}, Dep: {token.dep_}, Head: {token.head.text}")

        for token in doc:
            # Check if the token's lemma is 'be' or 'get'
            if token.lemma_ in auxiliary_lemmas:
                # Check if the main verb (past participle) is the head or a child of the auxiliary verb
                for child in token.head.children:
                    if child.pos_ == 'VERB' and child.tag_ == 'VBN':  # Past participle form
                        past_participles.append(child.text)  # Append the actual token text

                # Handle case where "get" is the main verb followed by a past participle
                if token.lemma_ == 'get' and token.dep_ == 'ROOT':  # Check if 'get' is the root
                    for child in token.children:
                        if child.pos_ == 'VERB' and child.tag_ == 'VBN':  # Past participle form
                            past_participles.append(child.text)

    return past_participles

# Example data (replace these with your actual data)
bepp_spoken = ["The book was written by the author.", "The letter is being delivered."]
bepp_written = ["The project was completed.", "The car was driven."]
getpp_spoken = ["He got promoted to manager.", "The boy got caught stealing."]
getpp_written = ["She got invited to the party.", "They got accepted to the university."]

# Extract past participles after 'be' and 'get'
bepp_spoken_pp = extract_past_participles(bepp_spoken, ['be'])
bepp_written_pp = extract_past_participles(bepp_written, ['be'])
getpp_spoken_pp = extract_past_participles(getpp_spoken, ['get'])
getpp_written_pp = extract_past_participles(getpp_written, ['get'])

# Output the extracted past participles
print("BePP Spoken Past Participles:", bepp_spoken_pp)
print("BePP Written Past Participles:", bepp_written_pp)
print("GetPP Spoken Past Participles:", getpp_spoken_pp)
print("GetPP Written Past Participles:", getpp_written_pp)
